In [1]:
from tracemalloc import start
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset
from datasets.dataset import *
# from models.Autoformer import *(我h'h'h)
from tqdm import tqdm
import argparse
import random
import math

from six.moves import cPickle as pickle

import scipy.stats
from scipy.signal import convolve2d
from scipy.signal import find_peaks
from sklearn.metrics import *

from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric
import os

from my_utils import *
from networks.mat import Finetune

from sklearn.metrics import roc_auc_score

from IPython import embed

import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
cuda_device = '0'
save_path = '/data/rech/dingqian/model_das/finetune_epoch_0'
scratch = True
ft_path = ['/data/rech/dingqian/data_das/finetune_epoch_0_train.npy', 
          '/data/rech/dingqian/data_das/finetune_epoch_0_test.npy']

# check save_path is exist, if not, create it
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
def check_mem(cuda_device):
    devices_info = os.popen('"/opt/bin/nvidia-smi" --query-gpu=memory.total,memory.used --format=csv,nounits,noheader').read().strip().split("\n")
    print(devices_info)
    total, used = devices_info[int(cuda_device)].split(',')
    return total,used

def occumpy_mem(cuda_device):
    total, used = check_mem(cuda_device)
    total = int(total)
    used = int(used)
    max_mem = int(total * 0.9)
    block_mem = max(0, max_mem - used)
    print(block_mem)
    x = torch.FloatTensor(256,1024,block_mem).to(f'cuda:{cuda_device}')
    del x
    
occumpy_mem(cuda_device)

['24564, 3881', '24564, 22813', '24564, 21957', '24564, 22813']
18226


In [4]:
with open(ft_path[0], 'rb') as f:
    x_ft_train, style_ft_train, skip_ft_train = pickle.load(f)
with open(ft_path[1], 'rb') as f:
    x_ft_test, style_ft_test, skip_ft_test = pickle.load(f)


In [5]:
picks_train = torch.load('/data/rech/dingqian/data_das/picks_train.pt')
patches_train = torch.load('/data/rech/dingqian/data_das/patches_train.pt')

picks_test = torch.load('/data/rech/dingqian/data_das/picks_test.pt')
patches_test = torch.load('/data/rech/dingqian/data_das/patches_test.pt')

In [6]:
'''finetune_new_train = [x_ft_train, style_ft_train]
finetune_new_test = [x_ft_test, style_ft_test]
with open('/data/rech/dingqian/data_das/finetune_new_train.npy', 'wb') as f:
    pickle.dump(finetune_new_train, f)

with open('/data/rech/dingqian/data_das/finetune_new_test.npy', 'wb') as f:
    pickle.dump(finetune_new_test, f)'''

"finetune_new_train = [x_ft_train, style_ft_train]\nfinetune_new_test = [x_ft_test, style_ft_test]\nwith open('/data/rech/dingqian/data_das/finetune_new_train.npy', 'wb') as f:\n    pickle.dump(finetune_new_train, f)\n\nwith open('/data/rech/dingqian/data_das/finetune_new_test.npy', 'wb') as f:\n    pickle.dump(finetune_new_test, f)"

In [7]:
# Convert picks to picks_filtered

def picks_filtered(picks, patches):
    picks_filtered = []
    patches_filtered = []

    for i, pick in enumerate(picks):
        if pick.sum().item() > 0:
            picks_filtered.append(pick)
            patches_filtered.append(patches[i])
    
    return torch.stack(picks_filtered), torch.stack(patches_filtered)

In [8]:
picks_train, patches_train = picks_filtered(picks_train, patches_train)
picks_test, patches_test = picks_filtered(picks_test, patches_test)

In [49]:
## picks_filtered to gaussian

def gaussian_2d(x=0, y=0, mx=0, my=0, sx=1, sy=1):
    # return 1 / (2*math.pi*sx*sy) * torch.exp(-((x - mx)**2 / (2*sx**2) + (y - my)**2 / (2*sy**2)))
    return torch.exp(-((x - mx)**2 / (2*sx**2) + (y - my)**2 / (2*sy**2)))

def tensor_to_hot(pick_oh_mat):
    
    pick_pos = torch.stack(torch.where(pick_oh_mat == 1), axis=1)
    
    h, w = 128, 128
    x = torch.linspace(0, h-1, h)
    y = torch.linspace(0, w-1, w)
    x, y = torch.meshgrid(x, y)

    z = torch.zeros(h, w)
    for x0, y0 in pick_pos:
        # z = torch.max(z, gaussian_2d(x, y, mx=x0, my=y0, sx=h/10, sy=w/10))
        z = (z + gaussian_2d(x, y, mx=x0, my=y0, sx=h/100, sy=w/200)).clip(max=1.)
    
    return z
    

In [10]:
### Get labels

def get_labels(picks_filtered):
    labels = []
    for i, pick in enumerate(tqdm(picks_filtered)):
        pick_oh = F.one_hot(pick.long(), 3)
        pick_p = tensor_to_hot(pick_oh[:, :, 1])
        pick_s = tensor_to_hot(pick_oh[:, :, 2])
        
        pick_n = (1 - pick_p - pick_s).clip(min=0.)
        
        pick_new = torch.stack([pick_n, pick_p, pick_s])
        labels.append(pick_new)
    
    return torch.stack(labels)

In [13]:
'''pick_oh = F.one_hot(picks_train[0].long(), 3)
%matplotlib inline
plt.imshow(tensor_to_hot(pick_oh[:, :, 1]), aspect='auto', vmin=-0, vmax=1.0, cmap="seismic")'''

In [10]:
labels_train = get_labels(picks_train)
labels_test = get_labels(picks_test)

    

  0%|                                                                                                                                                   | 0/2054 [00:00<?, ?it/s]/Tmp/dingqian/miniconda3/envs/ptpy3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2124/2124 [00:26<00:00, 80.18it/s]


In [12]:
labels_train.shape, labels_test.shape

NameError: name 'labels_train' is not defined

In [49]:
### Write a train function for each epoch

def train_loop(train_loader, model, optimizer, criterion, device):
    model.train()
    train_loss = 0
    for i, (x, style, skip, labels, picks) in enumerate(tqdm(train_loader)):
        x, style, skip, labels, picks = x.to(device), style.to(device), skip.to(device), labels.to(device), picks.to(device)
        optimizer.zero_grad()
        _, y = model(x, style, skip)
        y = y.softmax(dim=1)
        
        label_indices = torch.where(picks.sum(-1) > 0)
        labels = labels[label_indices[0], :, label_indices[1], :]
        y = y[label_indices[0], :, label_indices[1], :]

        loss = criterion(y, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    return train_loss / len(train_loader)


In [50]:
def test_loop(test_loader, model, criterion, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (x, style, skip, labels, picks) in enumerate(test_loader):
            x, style, skip, labels, picks = x.to(device), style.to(device), skip.to(device), labels.to(device), picks.to(device)
            _, y = model(x, style, skip)
            y = y.softmax(dim=1)
            
            label_indices = torch.where(picks.sum(-1) > 0)
            labels = labels[label_indices[0], :, label_indices[1], :]
            y = y[label_indices[0], :, label_indices[1], :]
            
            loss = criterion(y, labels)
            test_loss += loss.item()
    
    return test_loss / len(test_loader)

In [52]:
# Output a sample image with predicted labels and ground truth labels on training set

# get the first batch of data in train_loader

def plot_sample(model, data_loader, epoch, prefix, device):
    model.eval()
    with torch.no_grad():
        x, style, skip, labels, picks = next(iter(data_loader))
        x, style, skip, labels, picks = x.to(device), style.to(device), skip.to(device), labels.to(device), picks.to(device)
        _, y = model(x, style, skip)
        y = y.softmax(dim=1)

        y = y.cpu().numpy()
        labels = labels.cpu().numpy()
        x = x.cpu().numpy()

        fig, ax = plt.subplots(2, 2, figsize=(15, 5))
        ax[0][0].imshow(y[0][1], aspect='auto', vmin=-0, vmax=1.0, cmap="seismic")
        ax[0][1].imshow(labels[0][1], aspect='auto', vmin=-.0, vmax=1.0, cmap="seismic")

        ax[1][0].imshow(y[0][2], aspect='auto', vmin=-0, vmax=1.0, cmap="seismic")
        ax[1][1].imshow(labels[0][2], aspect='auto', vmin=-.0, vmax=1.0, cmap="seismic")

        # save fig to file
        fig.savefig(f'{save_path}/{prefix}_{epoch}.png', dpi=300)

        # clear fig ax
        plt.close(fig)

        print('Max y[1]: ', y[0][1].max(), 'Max y[2]: ', y[0][2].max())
        print('Max labels[1]: ', labels[0][1].max(), 'Max labels[2]: ', labels[0][2].max())

        ### x has 180 samples. using matplotlib to plot x in a grid of size 12x15

        fig, ax = plt.subplots(12, 15, figsize=(15, 12))
        for i in range(12):
            for j in range(15):
                ax[i][j].imshow(x[0][i*15+j], aspect='auto', vmin=-1.0, vmax=1.0, cmap="viridis")
                ax[i][j].axis('off')
                
        # save fig to file
        fig.savefig(f'{save_path}/{prefix}_x_{epoch}.png', dpi=300)

        # clear fig ax
        plt.close(fig)
    

In [53]:
# Convert numpy data to pytorch dataset

x_ft_train = torch.from_numpy(x_ft_train).float()
style_ft_train = torch.from_numpy(style_ft_train).float()
skip_ft_train = torch.from_numpy(skip_ft_train).float()

x_ft_test = torch.from_numpy(x_ft_test).float()
style_ft_test = torch.from_numpy(style_ft_test).float()
skip_ft_test = torch.from_numpy(skip_ft_test).float()


TypeError: expected np.ndarray (got Tensor)

In [27]:
# Create dataset for training and testing
# input data: x_ft, style_ft, skip_ft
# output data: labels

train_dataset = TensorDataset(x_ft_train, style_ft_train, skip_ft_train, labels_train, picks_train)
test_dataset = TensorDataset(x_ft_test, style_ft_test, skip_ft_test, labels_test, picks_test)

# Create dataloader for training and testing

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [29]:
x_ft_test.shape, style_ft_test.shape, skip_ft_test.shape, labels_test.shape, picks_test.shape

(torch.Size([2124, 180, 64, 64]),
 torch.Size([2124, 540]),
 torch.Size([2124, 180, 128, 128]),
 torch.Size([2124, 3, 128, 128]),
 torch.Size([2124, 128, 128]))

In [21]:
labels_test[0][1].sum(1)

tensor([1.1250e-07, 1.0281e-06, 8.0667e-06, 5.4336e-05, 3.1423e-04, 1.5603e-03,
        6.6534e-03, 2.4370e-02, 7.6714e-02, 2.0775e-01, 4.8496e-01, 9.7946e-01,
        1.7239e+00, 2.6797e+00, 3.6559e+00, 4.2686e+00, 4.8575e+00, 5.1070e+00,
        5.4112e+00, 5.6460e+00, 5.7238e+00, 5.7952e+00, 5.8434e+00, 5.8585e+00,
        5.8353e+00, 5.7713e+00, 5.6681e+00, 5.5354e+00, 4.9895e+00, 4.3113e+00,
        3.3278e+00, 1.8229e+00, 1.0301e+00, 9.0057e-01, 1.3705e+00, 2.4371e+00,
        3.9248e+00, 4.9152e+00, 5.5618e+00, 6.2929e+00, 6.8179e+00, 7.2794e+00,
        7.3275e+00, 7.2509e+00, 6.9057e+00, 6.2508e+00, 5.0894e+00, 3.3265e+00,
        1.7310e+00, 7.9016e-01, 3.1660e-01, 1.1743e-01, 6.4144e-02, 1.1078e-01,
        2.9073e-01, 7.0688e-01, 1.4994e+00, 2.7674e+00, 4.4335e+00, 5.3067e+00,
        5.7442e+00, 5.8055e+00, 5.6663e+00, 5.4018e+00, 4.9775e+00, 4.6554e+00,
        4.6154e+00, 4.8669e+00, 5.3493e+00, 5.8559e+00, 6.0805e+00, 6.2807e+00,
        6.3170e+00, 6.2342e+00, 6.0858e+

In [23]:
picks_test.shape

torch.Size([2124, 128, 128])

In [ ]:
labels_test[0][:, :, torch.where(picks_test[0].sum(-1) > 0)[0]]

In [34]:
torch.where(picks_test[0].sum(1) > 0)

(tensor([  0,  15,  16,  17,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,
          38,  40,  41,  42,  43,  44,  45,  56,  59,  60,  61,  63,  68,  70,
          71,  72,  73,  74,  76,  77,  78,  81,  82,  83,  84,  85,  87,  88,
          89,  90,  91,  92,  93,  94,  96,  97,  98,  99, 100, 101, 102, 103,
         104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
         120, 121, 122, 123, 127]),)

In [85]:
# Define model
import importlib
import networks.mat as mat
importlib.reload(mat)

device = torch.device(f'cuda:{cuda_device}')

model = mat.Finetune(img_channels=3).to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Define loss function as Softmax Cross Entropy
criterion = nn.CrossEntropyLoss()

# Define loss function as MSE loss

# criterion = nn.MSELoss()

['24564, 24243', '24564, 14433', '24564, 22813', '24564, 22113']
0


In [ ]:
# Train model and save best model on test set
try:
    model.load_state_dict(torch.load(f'{save_path}/last_model.pth'))
    epoch_start = 4797
except:
    epoch_start = 0
 
epochs = 4000
best_loss = np.inf
for epoch in range(epoch_start, epoch_start + epochs):
    train_loss = train_loop(train_loader, model, optimizer, criterion, device)
    test_loss = test_loop(test_loader, model, criterion, device)
    print(f'Epoch {epoch}: train loss {train_loss:.4f}, test loss {test_loss:.4f}')

    if epoch % 1 == 0:
        plot_sample(model, train_loader, epoch, 'train', device)
        plot_sample(model, test_loader, epoch, 'test', device)
    
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), f'{save_path}/best_model.pth')
    
    torch.save(model.state_dict(), f'{save_path}/last_model.pth')

In [ ]:
embed()